In [ ]:
import arcpy 
import requests
from urllib.parse import urlencode
import csv
import time

api_key = "<api_key_here>"

addresses_tbl_path  = r"L:\Work\STR_2023\STR_2023.gdb\address_tbl_20231218"
address_fields = ["STR_ID","Address","City","Code","State__Province_","Country"]  # input fields
output_csv_path = r"L:\Work\STR_2023\data\osm_results_raw.csv"

max_rows_to_process = 2000  # use during testing.  Set to > data volume for prod e.g. 99999

delay_seconds = 1  # how many seconds to wait between requests (for rate limiting)


In [ ]:
# Get ORS geocode
def geocode_address(api_key,address,locality,postalcode,region,country):
    
    # Add a delay of 1 second
    time.sleep(delay_seconds)

    # Base URL for the OpenRouteService structured geocode API
    base_url = "https://api.openrouteservice.org/geocode/search/structured"

    # Formatting the query parameters
    params = {
        "api_key": api_key,
        "address": address,
        "locality": locality,
        "postalcode": postalcode,
        "region": region,
        "country": country
    }

    # Encoding the parameters and forming the full URL
    full_url = f"{base_url}?{urlencode(params)}"

    # Making the request
    response = requests.get(full_url)
    print(response)
    print("-----------")
    
    # Checking if the request was successful
    if response.status_code == 200:
        geocoded_data = response.json()
        
        # Extracting the first feature's data
        first_feature = geocoded_data['features'][0]
        properties = first_feature['properties']

        # Extracting coordinates
        longitude, latitude = first_feature['geometry']['coordinates']

        # Extracting other relevant information
        source = properties['source']
        accuracy = properties['accuracy']
        match_type = properties['match_type']
        
        # Extracting matched address components
        matched_address = properties.get('street', 'N/A')
        matched_locality = properties.get('locality', 'N/A')
        matched_postalcode = properties.get('postalcode', 'N/A')
        matched_region = properties.get('region', 'N/A')
        matched_country = properties.get('country', 'N/A')

        return latitude, longitude, source, accuracy, match_type, matched_address, matched_locality, matched_postalcode, matched_region, matched_country
    else:
        return None, None, None, None, None, None, None, None, None, None


In [ ]:
# Main Loop

# Write CSV header
with open(output_csv_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['STR_ID', 'Latitude', 'Longitude', 'Source', 'Accuracy', 'Match_Type', 'Matched_Address', 'Matched_Locality', 'Matched_Postalcode', 'Matched_Region', 'Matched_Country'])

    n = 1
    # Using a search cursor to loop through the records
    with arcpy.da.SearchCursor(addresses_tbl_path, address_fields) as cursor:
        for row in cursor:
            str_id = row[0]
            address = row[1]
            locality = row[2]   # City
            postalcode = row[3] # Postcode
            region = row[4]     # State
            country = row[5]

            print("---------------------------------------------------------------------------------------------------------------------------")
            print(f"#{n}")
            print(f"STR_ID: {str_id}, Address: {address}, Locality: {locality}, Postcode: {postalcode}, Region: {region}, Country: {country}")
            latitude, longitude, source, accuracy, match_type, matched_address, matched_locality, matched_postalcode, matched_region, matched_country = geocode_address(api_key,address,locality,postalcode,region,country)

            # Write to CSV
            writer.writerow([str_id, latitude, longitude, source, accuracy, match_type, matched_address, matched_locality, matched_postalcode, matched_region, matched_country])
            
            if n >= max_rows_to_process: break
            n = n + 1

print("done")

